In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from garuda.od import ConfusionMatrix
from PIL import Image

how many random predictions -- should it be same as no of predictions per patch

should random predictions have mean and std dev same as original predictions


save the original predictions and then use it for getting mean and std dev

In [2]:
def add_class_confidence(predicted_results):
    new_predicted_results = []
    for res in predicted_results:
        if len(res):
            res = np.hstack([np.zeros((len(res),1)), res, np.ones((len(res),1))], dtype=np.float32) # add class label 0 at index 0 and confidence score 1 at last index
            new_predicted_results.append(res)
        else:
            res = np.zeros((1, 10))
            res[:, 0] = 1
            res[:, -1] = 1
            new_predicted_results.append(res.astype(np.float32))
    return new_predicted_results

In [3]:
def modify_class(target_results):
    new_target_results = []
    for res in target_results:
        res[:,0] = 0 # convert class labels to 0
        res = res.astype(np.float32)
        new_target_results.append(res)
    return new_target_results

In [16]:
def calculate_confusion_matrix(new_predicted_results, new_target_results, iou_threshold = 0.1):
    cm_predicted_results = []
    for res in new_predicted_results:
        res[:,1:9] = res[:,1:9]*500
        cm_predicted_results.append(res)


    cm_target_results = []
    for res in new_target_results:
        res[:,0] = 0 # convert class labels to 0
        res[:,1:9] = res[:,1:9]*500
        res = res.astype(np.float32)
        cm_target_results.append(res)

    classes, conf_threshold, iou_threshold = ['brick_kilns'], 0.25, iou_threshold
    cm = ConfusionMatrix.from_obb_tensors(cm_predicted_results, cm_target_results, classes, conf_threshold, iou_threshold)
    # cm = MyConfusionMatrix.from_tensors(cm_predicted_results, cm_target_results, classes, conf_threshold, iou_threshold)
    df = pd.DataFrame(cm.matrix, columns = ['predicted kilns','predicted_bg'], index=['true kilns','true_bg'])
    print(f'conf_threshold = {conf_threshold}, iou_threshold = {iou_threshold}')
    # print(cm.summary)
    # print(df.to_markdown())
    return cm, df

In [20]:
def calculate_precision_recall(confusion_matrix, new_predicted_results, new_target_results):
    tp = confusion_matrix.loc['true kilns']['predicted kilns']
    predicted_positives = 0
    for res in new_predicted_results:
        predicted_positives += np.where(res[:,0] == 1, 0, 1).sum() # class = 1 means background class
    
    ground_truth = 0
    for res in new_target_results:
        ground_truth += np.where(res[:,0] == 0, 1, 0).sum() # class = 0 means brick kiln class

    precision = tp / (predicted_positives + 1e-9)
    recall = tp/ground_truth
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)

    return precision, recall, f1_score

In [6]:
def plot_results(user_message, gr_imgs_path, new_predicted_results, new_target_results, image_path_with_detected_objects, region, plot=False):
    if not plot:
        return
    # n = len(image_path_with_detected_objects)
    gr_img_path = sorted(glob(gr_imgs_path+'/*'))
    n = len(gr_img_path)
    fig, ax = plt.subplots(nrows = n, ncols = 1 ,figsize=(120, 120))
    ax = ax.flatten()
    for i in range(n):
        # img = Image.open(image_path_with_detected_objects[i]).convert('RGB') # predicted image
        img = Image.open(gr_img_path[i]).convert('RGB') # planet image
        w, h = img.size
        # print(w, h)
        ax[i].imshow(img) 
        for bbox in new_target_results[i]:
            classvalue, x1, y1, x2, y2, x3, y3, x4, y4 = bbox*w 
            ax[i].plot([x1, x2, x3, x4, x1], [y1, y2, y3, y4, y1], color = 'green')
        for bbox in new_predicted_results[i]:
            classvalue, x1, y1, x2, y2, x3, y3, x4, y4, conf = bbox*w 
            ax[i].plot([x1, x2, x3, x4, x1], [y1, y2, y3, y4, y1], color = 'red')
        ax[i].set_axis_off()

    fig.suptitle(f'{user_message}')
    # region = region.replace('/','_')
    # plt.savefig(f'geochat_output_refer_{region}.png')
    # plt.close() # will not display the plot

prompt1 = '[refer] Where is the brick kiln with chimney in the image? Give its bounding box'

prompt2 = '[refer]_Where_are_the_factories_in_the_image_Give_its_oriented_bounding_box'

In [14]:
gr_imgs_paths = [
                '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/vlm_data/lucknow_airshed_most_15/images',
                '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/swinir_data/lucknow_airshed_most_15/images',
                '/home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_high_resolution_zoom17',
                '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/vlm_data/uttar_pradesh_most_15/images',
                '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/vlm_data/uttar_pradesh_most_15/swinir_images',
                '/home/shataxi.dubey/shataxi_work/vlm_on_planet/uttar_pradesh_high_resolution_zoom17',
                '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/vlm_data/west_bengal_most_15/images',
                '/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/vlm_data/west_bengal_most_15/swinir_images',
                '/home/shataxi.dubey/shataxi_work/vlm_on_planet/west_bengal_high_resolution_zoom17'
                ]

prompts = [
           'prompt1', 
           'prompt2'
           ]

gr_imgs_pred_labels_paths = [
                            '/home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_planet',
                            '/home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_swinir',
                            '/home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_zoom17',
                            '/home/shataxi.dubey/shataxi_work/vlm_on_planet/uttar_pradesh_most_15_planet',
                            '/home/shataxi.dubey/shataxi_work/vlm_on_planet/uttar_pradesh_most_15_swinir',
                            '/home/shataxi.dubey/shataxi_work/vlm_on_planet/uttar_pradesh_most_15_zoom17',
                            '/home/shataxi.dubey/shataxi_work/vlm_on_planet/west_bengal_most_15_planet',
                            '/home/shataxi.dubey/shataxi_work/vlm_on_planet/west_bengal_most_15_swinir',
                            '/home/shataxi.dubey/shataxi_work/vlm_on_planet/west_bengal_most_15_zoom17',
                            ] 
locations = [
            'lucknow_airshed_most_15',
            'lucknow_airshed_most_15', 
            'lucknow_airshed_most_15', 
            'uttar_pradesh_most_15',
            'uttar_pradesh_most_15',
            'uttar_pradesh_most_15', 
            'west_bengal_most_15',
            'west_bengal_most_15', 
            'west_bengal_most_15', 
            ]



In [15]:
for gr_imgs_path, gr_imgs_pred_labels_path, location in zip(gr_imgs_paths, gr_imgs_pred_labels_paths, locations):
    np.random.seed(5)
    
    for prompt in prompts:
        print(f'Path : {gr_imgs_pred_labels_path}, Prompt : {prompt}')
        path = f'{gr_imgs_pred_labels_path}/{prompt}/*'
        labels = sorted(glob(path))
        random_boxes = []
        for label in labels:
            boxes = np.loadtxt(label, ndmin = 2)
            w = boxes[:,[0]] - boxes[:,[2]]
            wmean, wstd = w.mean(), w.std()
            h = boxes[:,[5]] - boxes[:,[1]]
            hmean, hstd = h.mean(), h.std()
            
            tmp = []
            for _ in range(len(boxes)):
                width = abs(np.random.normal(wmean, wstd, 1)[0])
                height = abs(np.random.normal(hmean, hstd, 1)[0])
                x1, y1 = np.random.rand(2)
                # print(x1, y1)
                x2 = x1 + width if x1 + width < 1 else 1
                y2 = y1
                x3 = x2 
                y3 = y1 + height if y1 + height < 1 else 1
                x4, y4 = x1, y3
                # print(x1, y1, x2, y2, x3, y3, x4, y4)
                tmp.append(np.array([x1, y1, x2, y2, x3, y3, x4, y4]))
            random_boxes.append(np.array(tmp))
        # print(random_boxes)


        labels = sorted(glob(f'/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/vlm_data/{location}/labels/*'))

        actual_boxes = []
        for label in labels:
            boxes = np.loadtxt(label, ndmin = 2)
            actual_boxes.append(boxes)

        new_predicted_results = add_class_confidence(random_boxes)
        new_target_results = modify_class(actual_boxes)
        plot_results('user_message', gr_imgs_path, new_predicted_results, new_target_results, None, 'region', False)
        cm, df = calculate_confusion_matrix(new_predicted_results, new_target_results)
        # # print(f'Precision: {cm.precision}, Recall: {cm.recall}, F1 Score: {cm.f1_score}')
        precision, recall, f1_score = calculate_precision_recall(df, new_predicted_results, new_target_results)
        print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1_score}')
        print(df)

Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_planet, Prompt : prompt1
conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.0, Recall: 0.0, F1 Score: nan
            predicted kilns  predicted_bg
true kilns              0.0         155.0
true_bg               126.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_planet, Prompt : prompt2
conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.014814814814814815, Recall: 0.012903225806451613, F1 Score: 0.013793103448275864
            predicted kilns  predicted_bg
true kilns              2.0         153.0
true_bg               133.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_swinir, Prompt : prompt1
conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.006787330316742082, Recall: 0.07741935483870968, F1 Score: 0.012480499219968797
            predicted kilns  predicted_bg
true kilns             12.0         14

conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.006685236768802228, Recall: 0.07741935483870968, F1 Score: 0.012307692307692306
            predicted kilns  predicted_bg
true kilns             12.0         143.0
true_bg              1783.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_zoom17, Prompt : prompt2
conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.010095011876484561, Recall: 0.10967741935483871, F1 Score: 0.01848830886351278
            predicted kilns  predicted_bg
true kilns             17.0         138.0
true_bg              1667.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/uttar_pradesh_most_15_planet, Prompt : prompt1
conf_threshold = 0.25, iou_threshold = 0.1
Precision: nan, Recall: 0.0, F1 Score: nan
            predicted kilns  predicted_bg
true kilns              0.0           0.0
true_bg                 0.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/uttar_p

conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.026840490797546013, Recall: 0.13257575757575757, F1 Score: 0.044642857142857144
            predicted kilns  predicted_bg
true kilns             35.0         229.0
true_bg              1269.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/uttar_pradesh_most_15_zoom17, Prompt : prompt1
conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.010680157391793142, Recall: 0.07196969696969698, F1 Score: 0.01860009789525208
            predicted kilns  predicted_bg
true kilns             19.0         245.0
true_bg              1760.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/uttar_pradesh_most_15_zoom17, Prompt : prompt2
conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.014863258026159334, Recall: 0.0946969696969697, F1 Score: 0.025693730729701957
            predicted kilns  predicted_bg
true kilns             25.0         239.0
true_bg              1657.0           0.0
Path : /home/s

conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.013997760358342666, Recall: 0.12953367875647667, F1 Score: 0.025265285497726126
            predicted kilns  predicted_bg
true kilns             25.0         168.0
true_bg              1761.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/west_bengal_most_15_swinir, Prompt : prompt2
conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.013958125623130608, Recall: 0.07253886010362694, F1 Score: 0.023411371237458196
            predicted kilns  predicted_bg
true kilns             14.0         179.0
true_bg               989.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/west_bengal_most_15_zoom17, Prompt : prompt1
conf_threshold = 0.25, iou_threshold = 0.1
Precision: 0.007315700619020821, Recall: 0.06735751295336788, F1 Score: 0.013197969543147208
            predicted kilns  predicted_bg
true kilns             13.0         180.0
true_bg              1764.0           0.0
Path : /home/sha

# Calculate precision, recall at different IoU thresholds

In [21]:
for gr_imgs_pred_labels_path, location in zip(gr_imgs_pred_labels_paths, locations):    
    for prompt in prompts:
        for iou_threshold in [0.5]:
            print(f'Path : {gr_imgs_pred_labels_path}, Prompt : {prompt}, IoU threshold : {iou_threshold}')
            path = f'{gr_imgs_pred_labels_path}/{prompt}/*'
            labels = sorted(glob(path))
            predicted_boxes = []
            for label in labels:
                boxes = np.loadtxt(label, ndmin = 2)
                predicted_boxes.append(boxes)

            labels = sorted(glob(f'/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/vlm_data/{location}/labels/*'))

            actual_boxes = []
            for label in labels:
                boxes = np.loadtxt(label, ndmin = 2)
                actual_boxes.append(boxes)

            new_predicted_results = add_class_confidence(predicted_boxes)
            new_target_results = modify_class(actual_boxes)
            plot_results('user_message', gr_imgs_path, new_predicted_results, new_target_results, None, 'region', False)
            cm, df = calculate_confusion_matrix(new_predicted_results, new_target_results, iou_threshold)
            # # print(f'Precision: {cm.precision}, Recall: {cm.recall}, F1 Score: {cm.f1_score}')
            precision, recall, f1_score = calculate_precision_recall(df, new_predicted_results, new_target_results)
            print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1_score}')
            print(df)

Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_planet, Prompt : prompt1, IoU threshold : 0.5
conf_threshold = 0.25, iou_threshold = 0.5
Precision: 0.03174603174577979, Recall: 0.025806451612903226, F1 Score: 0.028469750394903827
            predicted kilns  predicted_bg
true kilns              4.0         151.0
true_bg               122.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_planet, Prompt : prompt2, IoU threshold : 0.5
conf_threshold = 0.25, iou_threshold = 0.5
Precision: 0.04444444444411523, Recall: 0.03870967741935484, F1 Score: 0.04137930984706303
            predicted kilns  predicted_bg
true kilns              6.0         149.0
true_bg               129.0           0.0
Path : /home/shataxi.dubey/shataxi_work/vlm_on_planet/lucknow_airshed_most_15_swinir, Prompt : prompt1, IoU threshold : 0.5
conf_threshold = 0.25, iou_threshold = 0.5
Precision: 0.01074660633483555, Recall: 0.12258064516129032, F1